# Kernel Implementation
### The Gaussian Smoothing function is used to smooth out the point values, extrapolating them to other positions.
 Given the distribution of particles, we can reconstruct the density at any location using the smoothing kernels. For example, the density at each SPH particle location is a sum over all the particles with weighting set by the distances between particles and the smoothing kernel:

In [41]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
from pytreegrav import Accel, Potential
#from ipynb.fs.full.tests import all_tests;

In [42]:
def W_Holtz(r, h):
	"""
    Gausssian Smoothing kernel (3D)
	x     is a vector/matrix of x positions
	y     is a vector/matrix of y positions
	z     is a vector/matrix of z positions
	h     is the smoothing length
	w     is the evaluated smoothing function
	"""
	w = (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2)
	return w
	

def dW_Holtz(r, h ):
	"""
	Gradient of the Gausssian Smoothing kernel (3D)
	x     is a vector/matrix of x positions
	y     is a vector/matrix of y positions
	z     is a vector/matrix of z positions
	h     is the smoothing length
	wx, wy, wz     is the evaluated gradient
	"""
	n = -2 * np.exp( -r**2 / h**2) / h**5 / (np.pi)**(3/2)
	return n

In [43]:
def apply_condition(Array,condition,function):
    Array[condition] = function(Array[condition])
    Array[~condition] = Array[~condition]
    return Array

def conditions(n, conditions_list):
    """ Takes in a numpy array, and a list of conditions and functions to use.
    Example usage: 
        n : np array [1,2,3,4]
        f = lambda x : 5
        f2 = lambda x : x+10
        L = [(n>2,f),(n<2,f2)]
    
        conditions(n,L)
        out: >> array([11,  2,  5,  5])
    """
    for i in conditions_list:
        out = apply_condition(n,i[0],i[1])
    return out

In [44]:
def cubic_spline_kernel(r,h,grad=False): 
    """ https://pysph.readthedocs.io/en/latest/reference/kernels.html"""
    zero = lambda x: 0
    sigma = 1/(np.pi*h**3)
    r = r/h
    if grad:
        cg1 = lambda x: sigma*(1-3*x*(1-x/2)-1.5*x**2*0.5)
        cg2 = lambda x: -1*sigma/4*2*(2-x)
    else:
        cg1 = lambda x: sigma*(1-1.5*x**2*(1-x/2))
        cg2 = lambda x: sigma/4*(2-x)**2
    L = [(r <= 1, cg1), (r<=2, cg2), (r > 2, zero),(r < 0, zero)]
    return conditions(r,L)
    
def starcrash_kernel(r,h,grad=False):
    zero = lambda x: 0
    r = r/h
    C = 1/(np.pi*h**3)
    if grad:
        cg1 = lambda x: C/h*(- 1.5*x*2 + 3*0.75*x**2)
        cg2 = lambda x: C/h*0.25*(-1)**3
    else:
        cg1 = lambda x: C*(1 - 1.5*x**2 + 0.75*x**3)
        cg2 = lambda x: C*0.25*(2-x)**3
    L = [(r<=2, cg2), (r <= 1, cg1),  (r > 2, zero),(r < 0, zero)]
    return conditions(r,L)

### ========= WENDLAN KERNEL (ROSSWOG, 2015) ========= ###  
    # Gradient of WendlandQuintic kernel(C2) for 3D.         # https://pysph.readthedocs.io/en/latest/reference/kernels.html# Wendland C4
    # Gradient of WendlandQuintic kernel(C4) for 3D.          # https://pysph.readthedocs.io/en/latest/reference/kernels.html
    
def wend2_kernel(r,h,grad=False):
    r = r/h
    zero = lambda x: 0
    if grad:
        alpha = 7/(4*np.pi*h**2)
        cg1 = lambda x: alpha*(-2*(1-x/2)**3*(2*x+1) +2*(1-x/2)**4)
    else:
        alpha = 21/(16*np.pi*h**3)
        cg1 = lambda x: alpha*(1-x/2)**4*(2*r+1)
    L = [(r<=2, cg1), (r > 2, zero),(r < 0, zero)]
    return conditions(r,L)
    
def wend4_kernel(r,h,grad=False):
    r = r/h
    zero = lambda x: 0
    if grad:
        alpha = 495/(256*np.pi*h**3) ## Take derivative wrt r
        cg1 = lambda x: alpha*-2.5*(1-x/2)**5*(35/12*x**2 + 3*x + 1) + (1-x/2)**6*(35/12*x +3) ## Take derivative wrt r
    else:
        alpha = 495/(256*np.pi*h**3)
        cg1 = lambda x: alpha*(1-x/2)**6*(35/12*x**2 + 3*x + 1)
    L = [(r<=2, cg1), (r > 2, zero),(r < 0, zero)]
    return conditions(r,L)

def holtz_gauss_kernel(r,h,grad=False):
    if grad:
        return dW_Holtz(r,h)
    else:
        return W_Holtz(r,h)

In [45]:
get_sphpy_kernels = lambda tag: print("No module named 'pysph'")
# import pysph

# def get_sphpy_kernels(tag):
#    """ https://pysph.readthedocs.io/en/latest/reference/kernels.html"""
#     if tag = "cubic_spline":
#         return pysph.base.kernels.CubicSpline(dim=3)
#     if tag == "gaussian": 
#         return pysph.base.kernels.Gaussian(dim=3) 
#     if tag ==   "quinticspline":
#         return pysph.base.kernels.QuinticSpline(dim=3)
#     if tag == "super_gaussian": 
#         return pysph.base.kernels.SuperGaussian(dim=3)
#     if tag == "wend2":
#         return pysph.base.kernels.WendlandQuintic(dim=3)
#     if tag == "wend4" :
#         return pysph.base.kernels.WendlandQuinticC4(dim=3)
#     if tag == "wend6": 
#         return pysph.base.kernels.WendlandQuinticC6(dim=3)

### Kernel Types Implementation

### Select Kernel

In [46]:
def get_kernel(tag):
    if tag == "gaussian":
        return holtz_gauss_kernel
    elif tag == "wendland_2": 
        return wend2_kernel
    elif tag == "wendland_4": 
        return wend4_kernel
    elif tag == "cubic_spline":
        return cubic_spline_kernel
    elif tag == "faber":
        return starcrash_kernel
    elif tag == "gaussian_s":
        return holtz_gauss_kernel
    else:
        raise NotImplementedError("Trying to implement Unknown Kernel.")

### Define Kernel

In [47]:
def W(x,y,z,h, tag = "gaussian",sphpy=False):
    """ Defined Smoothing kernel (3D).
        (x is a vector/matrix of x positions, y is a vector/matrix of y positions. 
        z = vector/matrix of z positions, h = smoothing length, w = evaluated smoothing function.
        Implemented Gaussian, Wendland C2 and Wendland C4 smoothing kernels as recommended by (Rosswog, 2015). """
    r = np.sqrt(x**2 + y**2 + z**2) 
    if sphpy:
        kernel = get_sphpy_kernels(tag)
    else:
        kernel = get_kernel(tag)
    #kernel = np.vectorize(kernel) # convert the function which takes in integers to a funciton that can take np arrays.
    return kernel(r,h) # retrun the kernel evaluated at r with smoothing length h.

### Calculate Gradient of the Kernel

In [48]:
def gradW(x,y,z,h,tag = "gaussian",sphpy=False):
    """ Gradient of smoothing kernel. We can reconstruct the density at any location using the smoothing kernels.
    (x is a vector/matrix of x positions, y is a vector/matrix of y positions, z is a vector/matrix of z positions, h is the smoothing length, wx, wy, wz  is the evaluated gradient)"""
    r = np.sqrt(x**2 + y**2 + z**2)
    if sphpy: 
        kernel = get_sphpy_kernels(tag)
    else:
        kernel = get_kernel(tag)
    #kernel = np.vectorize(kernel) # convert the function which takes in integers to a funciton that can take np arrays.
    n = kernel(r,h,grad=True) # gradient of the kernel evaluated at r with smoothing length h.
    return n*x, n*y, n*z  # (gradient in the x, y, and z directions)

# TESTING

In [49]:
gradW(1,1,1,1)

(-0.017882232654467207, -0.017882232654467207, -0.017882232654467207)